**Modelo predictivo: recomendación académica de los estudiantes de la Universidad Metropolitana**

Universidad Metropolitana

Proyecto de Investigación de Ingeniería de Sistemas

## Imports

In [0]:
from google.colab import drive

import pandas as pd
import numpy as np
import os 
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dropout, Dense, Flatten, Activation, LSTM, Bidirectional, Concatenate
from keras.models import Model

# Para almacenar y obtener el modelo para predicciones futuras de un archivo .pkl.
from sklearn.externals import joblib

#Métrica de AUC
from sklearn.metrics import roc_auc_score

from IPython.display import HTML, display
import time

## Montar Drive

In [0]:
drive.mount('/content/drive', force_remount=True)

## Cargar el One Hot Encoding realizado

In [0]:
df = pd.read_csv('drive/Shared drives/Tesis/Proyecto de investigación/Modelo 5/Datos/one_hot_new_classes.csv', encoding='latin-1')

## Agrupar estudiantes y ordenar por trimestres

In [0]:
grouped = df.sort_values('trimestre').groupby(['estudiante'])

In [0]:
ids = grouped['estudiante'].unique().values

train_ids, test_ids = train_test_split(ids, test_size=0.05)

train = pd.DataFrame()
test = pd.DataFrame()
num=0

for iid in train_ids:
  train = train.append(df.loc[df['estudiante'] == iid[0]])

for iid in test_ids:
  test = test.append(df.loc[df['estudiante'] == iid[0]])

In [0]:
grouped = train.sort_values('trimestre').groupby(['estudiante'])
grouped_test = test.sort_values('trimestre').groupby(['estudiante']) 

## Separando el data set

Se separa el dataset en train, dev y test. Luego, se crean las variables X, Y y O con las que se van a entrenar al modelo.

**Auxiliar para mostrar el avance**

In [0]:
def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

**Estableciendo el diccionario de asignaturas**

In [0]:
# Contains all codes for asignatures in pensum of System Engineer
all_assigns = ['FBTCE03','FBTMM00','FBTHU01','FBTIE02','BPTQI21','BPTMI04','FBPIN01'
,'BPTPI07','FBPLI02','FBTIN04','FGE0000','FBPCE04','FBPMM02','FBTIN05'
,'FBPIN03','FBPIN02','FBPLI01','FBPCE03','FBPMM01','FBTHU02','FBTSP03'
,'BPTFI02','BPTMI11','BPTSP05','BPTMI01','FBTCE04','FBTMM01','FGS0000'
,'FBTIE03','BPTFI03','BPTMI20','BPTFI01','BPTQI22','BPTMI05','BPTMI30'
,'BPTSP06','BPTMI02','BPTMI03','FPTCS16','FPTSP15','BPTEN12','BPTMI31'
,'FPTEN23','BPTSP03','BPTFI04','FPTSP14','BPTDI01-1','FBTIE01','FPTSP20'
,'FPTMI21','BPTSP04','FPTSP01','FPTSP18','FPTSP22','FPTSP17','FPTPI09'
,'FPTSP11','FPTSP04','FPTSP02','BPTDI01-2','FPTSP23','FPTSP19','FPTSP07'
,'FPTSP25','FPTSP21','FPTIS01']

all_assigns.sort()

**Data augmentation del conjunto de entrenamiento**


*   Se realiza el aumentado de datos del conjunto pasado como entrada, tomando para cada estudiante conjuntos de trimestres de distinta longitud.
*   Se obtienen dos conjuntos: X (historial del estudiante), y Y (materias a inscribir por el estudiante)

In [0]:
array_data = [] # Contains the history of trimestres
array_target=[] # The target trimestres
array_target_output=[] # Output that specifies if the target is posible (all passed) or not

# Máximos de numero de trimestre y número de asignaturas
max_number_trim = grouped.count()['trimestre'].max()
max_number_assigns = 10

# Variables para el loader
out_estudiante = display(progress(0, len(grouped)), display_id=True)
i_estudiante = 0
out_trim = display(progress(0, len(grouped)), display_id=True)

# For para recorrer cada grupo de estudiantes
for est, est_group in grouped:
  i_estudiante += 1 # variable para el loader
  out_estudiante.update(progress(i_estudiante, len(grouped))) # progreso del loader
  count = 0 # contador loader
  
  for cant_trim in range(est_group.shape[0]):
    count += 1 # contador loader
    out_trim.update(progress(count, est_group.shape[0])) # progreso loader
    
    # Inicialización de row
    row = {}
    row_trim = []
    
    # Validador si la cantidad de trimestres (proveniente del for) es 0
    if cant_trim == 0:
      continue
    
    # Rellenar todos los espacios de trimestres con "asignaturas con notas" en 0 
    for num_empty in range(max_number_trim):
      row_trim.append(np.zeros((264), dtype=int))
    
    # Variable para determinar donde empiezan a llenarse con trimestres vistos 
    start_trim = max_number_trim - cant_trim
    
    # For para establecer valores de trimestres
    for num in range(cant_trim): 
      if num == cant_trim:
        continue
      row_trim[start_trim + num] = est_group.iloc[ num , 3:].values
      
    # Anexa filas con los trimestres vistos en cada una a array_data
    array_data.append(np.asarray(row_trim))
    
    # Obtiene las materias que el estudiante verá en el trimestre target
    assigns_trim_target = est_group.iloc[num+1, 3:].index[est_group.iloc[num+1, 3:] == 1].values # Obtain only assginatures with grades that the student have 
    
    # Arma el array_target
    only_assigns = {}
    
    # Establece en 0 todas las posiciones de las materias
    for assign_zero in all_assigns:
      only_assigns[assign_zero] = 0
    
    # Establece 1 en los códigos de las materias donde el estudiante verá materias en el trim_target
    for assign in assigns_trim_target:
      only_assigns[assign.split('_')[0]] = 1

    # Arma el array_target
    array_target.append(np.array( tuple(only_assigns.values()) ))  
  
    # Arma el array_target_output
    array_target_output.append(est_group.iloc[ num+1 , 3:].values)
    

**Estableciendo el target para cada entrada**

Se crea un array de targets, en el que para cada row del array de trimestres objetivo (target): 

*   Se asigna 1 si se obtuvo una calificación buena (mayor o igual a 13) en todas las materias. 
*   Se asigna 0 si reprobó/retiró u obtuvo una nota menor a 13 en alguna materia.

In [0]:
# Se crea el array de targets que tiene 0 si el estudiante reprobo, retiró u obtuvo una nota menor a 13 en alguna materia del trimestre target, y 1 en caso contrario.

arr_target = np.asarray(array_target_output)

sigm_target = np.zeros((arr_target.shape[0], 1), dtype=int)

for idx,item in enumerate(arr_target):
  columns = 4
  aux, mal, bien, reprobo, retiro = 0, 0, 0, 0, 0
  for one in range(int(item.shape[0]/4)):
    for col in range(columns):
      if item[col+columns*aux] == 1:
        if col == 0:
          bien += 1
        elif col == 1:
          mal += 1
        elif col == 2:
          retiro += 1
        elif col == 3:
          reprobo += 1
    aux += 1
  
  # Si no hay alguna materia reprobada o retirada, se asigna 1
  if reprobo == 0 and retiro == 0 :
    sigm_target[idx] = 1


**Separando el conjunto de datos de entrenamiento en los conjuntos de train y dev** 

(para el entrenamiento y validación del modelo)

In [0]:
# Se separa el dataset en la proporción 80-20-20 para train, dev y test respectivamente
array_data = np.asarray(array_data)
array_target = np.asarray(array_target)

X_train, X_dev, Y_train, Y_dev, O_train, O_dev = train_test_split(array_data, array_target, sigm_target, test_size=0.25)  # Sería el equivalente al 20% de toda la data


## Definición del modelo

Se construye un modelo que recibe dos entradas, un array del histórico del estudiante (X) y otro del trimestre objetivo o target (Y), y un output tipo sigmoide que indica 1 si el estudiante pasará todas las materias, y 0 en caso contrario (O).

In [0]:
def model():
  # Se definen dos inputs
  main_input = Input(shape=(max_number_trim,264), name='main_input')
  second_input = Input(shape=(66,), name='second_input')
  
  # La primera rama utiliza el main_input para la LSTM
  X = LSTM(32, return_sequences=False, dropout=0.1, recurrent_dropout=0.1)(main_input)
  X = Dense(32, activation='relu')(X)
  X = Dropout(0.2)(X)
  X = Dense(32, activation='relu')(X)
  X = Dropout(0.7)(X)
  X = Dense(32, activation='relu')(X)
  X = Dropout(0.1)(X)
  X = Dense(32, activation='relu')(X)
  
  # La segunda rama utiliza el second_input
  Y = Dense(32, activation='relu')(second_input)

  # Se combinan los outputs de las dos ramas
  combined = Concatenate()([X, Y])
  main_output = Dense(1, activation='sigmoid', name='main_output')(combined)
  
  model = Model(inputs=[main_input, second_input], outputs=[main_output])
  
  # Compile the model
  model.compile(
      optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])

  history = model.fit([X_train, Y_train],  O_train, 
                      batch_size=16, epochs=5,
                      validation_data=([X_dev, Y_dev], O_dev))

  return (model, history)

In [0]:
model, history = model()

## Graficar modelo

In [0]:
import matplotlib.pyplot as plt

def graf_model(train_history):
    f = plt.figure(figsize=(15,10))
    ax = f.add_subplot(121)
    ax2 = f.add_subplot(122)
    # summarize history for accuracy
    ax.plot(train_history.history['binary_accuracy'])
    ax.plot(train_history.history['val_binary_accuracy'])
    ax.set_title('model accuracy')
    ax.set_ylabel('accuracy')
    ax.set_xlabel('epoch')
    ax.legend(['train', 'test'], loc='upper left')
    # summarize history for loss
    ax2.plot(train_history.history['loss'])
    ax2.plot(train_history.history['val_loss'])
    ax2.set_title('model loss')
    ax2.set_ylabel('loss')
    ax2.set_xlabel('epoch')
    ax2.legend(['train', 'test'], loc='upper left')
    plt.show()

In [0]:
graf_model(history) 

## Test del modelo

In [0]:
def test_augmentation(grouped):  
  array_data = [] # Contains the history of trimestres
  array_target=[] # The target trimestres
  array_target_output=[] # Output that specifies if the target is posible (all passed) or not

  # Variables para el loader
  out_estudiante = display(progress(0, len(grouped)), display_id=True)
  i_estudiante = 0
  out_trim = display(progress(0, len(grouped)), display_id=True)

  # For para recorrer cada grupo de estudiantes
  for est, est_group in grouped:
    i_estudiante += 1 # variable para el loader
    out_estudiante.update(progress(i_estudiante, len(grouped))) # progreso del loader
    count = 0 # contador loader
    
    for cant_trim in range(est_group.shape[0]):
      count += 1 # contador loader
      out_trim.update(progress(count, est_group.shape[0])) # progreso loader
      
      # Inicialización de row
      row = {}
      row_trim = []
      
      # Validador si la cantidad de trimestres (proveniente del for) es 0
      if cant_trim == 0:
        continue
      
      # Rellenar todos los espacios de trimestres con "asignaturas con notas" en 0 
      for num_empty in range(max_number_trim):
        row_trim.append(np.zeros((264), dtype=int))
      
      # Variable para determinar donde empiezan a llenarse con trimestres vistos 
      start_trim = max_number_trim - cant_trim
      
      # For para establecer valores de trimestres
      for num in range(cant_trim): 
        if num == cant_trim:
          continue
        row_trim[start_trim + num] = est_group.iloc[ num , 3:].values

      # Anexa filas con los trimestres vistos en cada una a array_data
      array_data.append(np.asarray(row_trim))
      
      # Obtiene las materias que el estudiante verá en el trimestre target
      assigns_trim_target = est_group.iloc[num+1, 3:].index[est_group.iloc[num+1, 3:] == 1].values # Obtain only assginatures with grades that the student have 
      
      # Arma el array_target
      only_assigns = {}
      
      # Establece en 0 todas las posiciones de las materias
      for assign_zero in all_assigns:
        only_assigns[assign_zero] = 0
      
      # Establece 1 en los códigos de las materias donde el estudiante verá materias en el trim_target
      for assign in assigns_trim_target:
        only_assigns[assign.split('_')[0]] = 1
      # Arma el array_target
      array_target.append(np.array( tuple(only_assigns.values()) ))  
    
      # Arma el array_target_output
      array_target_output.append(est_group.iloc[ num+1 , 3:].values)

  return (array_data, array_target, array_target_output)

In [0]:
def set_sigm_test(array_target_output, ):
  # Se crea el array de targets que tiene 0 si el estudiante reprobo o retiro alguna materia en el trimestre target, y 1 en caso contrario

  arr_target = np.asarray(array_target_output)
  
  sigm_target = np.zeros((arr_target.shape[0], 1), dtype=int)

  for idx,item in enumerate(arr_target):
    columns = 4
    aux, mal, bien, reprobo, retiro = 0, 0, 0, 0, 0
    
    for one in range(int(item.shape[0]/4)):
      for col in range(columns):
        if item[col+columns*aux] == 1:
          if col == 0:
            bien += 1
          elif col == 1:
            mal += 1
          elif col == 2:
            retiro += 1
          elif col == 3:
            reprobo += 1
      aux += 1
    
    # Si no hay alguna materia reprobada o retirada, se asigna 1
    if reprobo == 0 and retiro == 0 :
      sigm_target[idx] = 1

  return sigm_target


In [0]:
X_test, Y_test, O_test = test_augmentation(grouped_test)
O_test =  set_sigm_test(O_test)

*Cargar archivos para probar de forma manual*

*Predicciones del conjunto de prueba*

In [0]:
def prediction():
  Y_hat_train = model.predict([X_train, Y_train])
  train_auc = roc_auc_score(O_train, Y_hat_train)
  
  Y_hat_dev = model.predict([X_dev, Y_dev])
  dev_auc = roc_auc_score(O_dev, Y_hat_dev)
  
  Y_hat_test = model.predict([X_test, Y_test])
  test_auc = roc_auc_score(O_test, Y_hat_test)
  
  return (Y_hat_train, Y_hat_dev, Y_hat_test)

In [0]:
Y_hat_train, Y_hat_dev, Y_hat_test = prediction()

## Referencias


https://www.pyimagesearch.com/2019/02/04/keras-multiple-inputs-and-mixed-data/

https://keras.io/getting-started/functional-api-guide/

https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.tolist.html